c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Without Symbol File

In [ ]:
# corrected_hedgeone_agent.py
# --- 1. IMPORTS (All in one place) ---
import os
import json
import numpy as np
import pandas as pd
import backtrader as bt
from dotenv import load_dotenv
from typing import List, Dict, Any
from datetime import datetime

# Fyers
from fyers_apiv3 import fyersModel

# LangChain (modern)
from langchain_groq import ChatGroq
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# RAG / Vector Store
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings

# --- 2. API & CONFIGURATION ---
load_dotenv()
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
FYERS_CLIENT_ID = os.environ.get("FYERS_CLIENT_ID")
FYERS_TOKEN = os.environ.get("FYERS_TOKEN")

# Global Fyers Model
fyers = fyersModel.FyersModel(
    client_id=FYERS_CLIENT_ID,
    token=FYERS_TOKEN,
    is_async=False,
    log_path=""
)

# Global RAG Config
VECTOR_STORE_PATH = "faiss_index_proto"

# --- 3. RAG METADATA (Replaces strategy_metadata.json) ---
STRATEGY_METADATA_LIST = [
    {
        "strategy_id": "SmaCrossStrategy",
        "description": "A simple trend-following strategy. It buys when a short-term moving average (n1) crosses above a long-term one (n2) and sells on the reverse cross. Best for simple, trending markets. Also known as a 'Golden Cross' or 'Death Cross'.",
        "parameters": [
            {"name": "n1", "type": "int", "description": "The period for the fast moving average, e.g., 50"},
            {"name": "n2", "type": "int", "description": "The period for the slow moving average, e.g., 200"}
        ]
    },
    {
        "strategy_id": "RsiStrategy",
        "description": "A mean-reversion strategy. It buys when the Relative Strength Index (RSI) crosses below an 'oversold' level (e.g., 30) and sells when it crosses above an 'overbought' level (e.g., 70).",
        "parameters": [
            {"name": "period", "type": "int", "description": "The lookback period for the RSI, typically 14."},
            {"name": "oversold", "type": "int", "description": "The RSI level considered oversold, typically 30."},
            {"name": "overbought", "type": "int", "description": "The RSI level considered overbought, typically 70."}
        ]
    },
    {
        "strategy_id": "MultiInstrumentSignal",
        "description": "A complex multi-asset strategy. It trades one target instrument (e.g., Nifty) based on the average performance of a basket of other 'signal' instruments (e.g., 10 stocks). All signal symbols must be provided first, and the target trade symbol last.",
        "parameters": []
    },
    {
        "strategy_id": "BollingerBandsReversion",
        "description": "A mean-reversion strategy. It buys when the price touches or crosses below the lower Bollinger Band and sells when it touches or crosses above the upper Bollinger Band.",
        "parameters": [
            {"name": "period", "type": "int", "description": "The lookback period for the moving average, typically 20."},
            {"name": "devfactor", "type": "float", "description": "The number of standard deviations for the bands, typically 2.0."}
        ]
    },
    {
        "strategy_id": "MACDStrategy",
        "description": "A trend-following strategy based on the Moving Average Convergence Divergence (MACD). It buys when the MACD line crosses above the signal line and sells when it crosses below.",
        "parameters": [
            {"name": "fast_ema", "type": "int", "description": "The period for the fast EMA, typically 12."},
            {"name": "slow_ema", "type": "int", "description": "The period for the slow EMA, typically 26."},
            {"name": "signal_ema", "type": "int", "description": "The period for the signal line EMA, typically 9."}
        ]
    },
    {
        "strategy_id": "StochasticStrategy",
        "description": "A momentum oscillator strategy. It buys when the %K line crosses above the %D line in the oversold region (e.g., below 20) and sells when it crosses below in the overbought region (e.g., above 80).",
        "parameters": [
            {"name": "k_period", "type": "int", "description": "The lookback period for %K, typically 14."},
            {"name": "d_period", "type": "int", "description": "The smoothing period for %D, typically 3."},
            {"name": "oversold", "type": "int", "description": "The oversold level, typically 20."},
            {"name": "overbought", "type": "int", "description": "The overbought level, typically 80."}
        ]
    },
    {
        "strategy_id": "DonchianChannelBreakout",
        "description": "A trend-following breakout strategy (like Turtle Trading). It buys when the price breaks above the upper channel (N-period high) and sells when it breaks below the lower channel (N-period low).",
        "parameters": [
            {"name": "period", "type": "int", "description": "The lookback period for the channel, typically 20."}
        ]
    },
    {
        "strategy_id": "EmaCrossStrategy",
        "description": "A simple trend-following strategy using Exponential Moving Averages (EMAs), which are faster to react than SMAs. Buys when the fast EMA (n1) crosses above the slow EMA (n2).",
        "parameters": [
            {"name": "n1", "type": "int", "description": "The period for the fast EMA, e.g., 12."},
            {"name": "n2", "type": "int", "description": "The period for the slow EMA, e.g., 26."}
        ]
    },
    {
        "strategy_id": "ATRTrailingStopStrategy",
        "description": "A trend-following strategy that uses an Average True Range (ATR) based trailing stop-loss. It buys on a signal (e.g., new high) and holds until the price crosses below the trailing stop.",
        "parameters": [
            {"name": "atr_period", "type": "int", "description": "The lookback period for the ATR, typically 14."},
            {"name": "atr_multiplier", "type": "float", "description": "The multiplier for the ATR value, e.g., 3.0."}
        ]
    },
    {
        "strategy_id": "OpeningRangeBreakout",
        "description": "An intraday strategy. It buys if the price breaks above the high of the first N minutes (e.g., 15) and sells/shorts if it breaks below the low. (Note: Requires intraday data).",
        "parameters": [
            {"name": "minutes", "type": "int", "description": "The opening range period in minutes, e.g., 15 or 30."}
        ]
    }
]

# --- 4. RAG SETUP FUNCTIONS ---
def create_vector_store():
    """
    Creates and saves a FAISS vector store from the strategy metadata.
    """
    print("Loading strategy metadata...")
    documents = []
    for strategy in STRATEGY_METADATA_LIST:
        content = f"Strategy: {strategy['strategy_id']}. Description: {strategy['description']}"
        metadata = {
            "strategy_id": strategy['strategy_id'],
            "description": strategy['description'],
            "parameters": json.dumps(strategy['parameters'])
        }
        documents.append(Document(page_content=content, metadata=metadata))
    
    print("Creating embeddings... (This may take a moment on first run)")
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vector_store = FAISS.from_documents(documents, embeddings)
    
    vector_store.save_local(VECTOR_STORE_PATH)
    print(f"Vector store saved to {VECTOR_STORE_PATH}")

def get_retriever():
    """
    Loads the saved FAISS vector store as a retriever.
    """
    if not os.path.exists(VECTOR_STORE_PATH):
        print("Vector store not found. Creating a new one...")
        create_vector_store()
        
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    # allow_dangerous_deserialization kept for compatibility with older saved indexes
    vector_store = FAISS.load_local(VECTOR_STORE_PATH, embeddings, allow_dangerous_deserialization=True)
    return vector_store.as_retriever(search_kwargs={"k": 1})

# --- 5. BACKTESTING STRATEGY CLASSES (copy your classes exactly) ---
class SmaCross(bt.Strategy):
    params = (('n1', 20), ('n2', 50))
    def __init__(self):
        self.sma_fast = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.n1)
        self.sma_slow = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.n2)
        self.crossover = bt.indicators.CrossOver(self.sma_fast, self.sma_slow)
    def next(self):
        if not self.position and self.crossover > 0: self.buy()
        elif self.position and self.crossover < 0: self.sell()

class RsiStrategy(bt.Strategy):
    params = (('period', 14), ('oversold', 30), ('overbought', 70))
    def __init__(self):
        self.rsi = bt.indicators.RSI(self.data.close, period=self.params.period)
    def next(self):
        if not self.position and self.rsi < self.params.oversold: self.buy()
        elif self.position and self.rsi > self.params.overbought: self.sell()

class MultiInstrumentSignal(bt.Strategy):
    def __init__(self):
        self.trade_data = self.datas[-1]
        self.signal_datas = self.datas[:-1]
    def next(self):
        returns = []
        for d in self.signal_datas:
            if len(d) > 1:
                daily_return = (d.close[0] - d.close[-1]) / d.close[-1]
                returns.append(daily_return)
        if not returns: return
        avg_return = np.mean(returns)
        if avg_return < -0.005 and not self.getposition(self.trade_data): self.sell(data=self.trade_data)
        elif avg_return > 0.005 and not self.getposition(self.trade_data): self.buy(data=self.trade_data)
        elif abs(avg_return) < 0.001 and self.getposition(self.trade_data): self.close(data=self.trade_data)

class BollingerBandsReversion(bt.Strategy):
    params = (('period', 20), ('devfactor', 2.0))
    def __init__(self):
        self.bbands = bt.indicators.BollingerBands(self.data.close, period=self.params.period, devfactor=self.params.devfactor)
    def next(self):
        if not self.position and self.data.close < self.bbands.lines.bot: self.buy()
        elif self.position and self.data.close > self.bbands.lines.top: self.sell()

class MACDStrategy(bt.Strategy):
    params = (('fast_ema', 12), ('slow_ema', 26), ('signal_ema', 9))
    def __init__(self):
        self.macd = bt.indicators.MACD(self.data.close, period_me1=self.params.fast_ema, period_me2=self.params.slow_ema, period_signal=self.params.signal_ema)
        self.crossover = bt.indicators.CrossOver(self.macd.macd, self.macd.signal)
    def next(self):
        if not self.position and self.crossover > 0: self.buy()
        elif self.position and self.crossover < 0: self.sell()

class StochasticStrategy(bt.Strategy):
    params = (('k_period', 14), ('d_period', 3), ('oversold', 20), ('overbought', 80))
    def __init__(self):
        self.stoch = bt.indicators.Stochastic(self.data, period=self.params.k_period, period_d=self.params.d_period)
    def next(self):
        if not self.position and self.stoch.percK < self.params.oversold and self.stoch.percD < self.params.oversold and self.stoch.percK > self.stoch.percD:
            self.buy()
        elif self.position and self.stoch.percK > self.params.overbought and self.stoch.percD > self.params.overbought and self.stoch.percK < self.stoch.percD:
            self.sell()

class DonchianChannelBreakout(bt.Strategy):
    params = (('period', 20),)
    def __init__(self):
        self.donchian = bt.indicators.DonchianChannels(self.data, period=self.params.period)
    def next(self):
        if not self.position and self.data.close[0] > self.donchian.dch[0]: self.buy()
        elif self.position and self.data.close[0] < self.donchian.dcl[0]: self.sell()

class EmaCrossStrategy(bt.Strategy):
    params = (('n1', 12), ('n2', 26))
    def __init__(self):
        self.ema_fast = bt.indicators.EMA(self.data.close, period=self.params.n1)
        self.ema_slow = bt.indicators.EMA(self.data.close, period=self.params.n2)
        self.crossover = bt.indicators.CrossOver(self.ema_fast, self.ema_slow)
    def next(self):
        if not self.position and self.crossover > 0: self.buy()
        elif self.position and self.crossover < 0: self.sell()

class ATRTrailingStopStrategy(bt.Strategy):
    params = (('atr_period', 14), ('atr_multiplier', 3.0))
    def __init__(self):
        self.atr = bt.indicators.ATR(self.data, period=self.params.atr_period)
        self.trailing_stop = 0
        self.is_long = False
    def next(self):
        if not self.is_long:
            if self.data.close > bt.indicators.Highest(self.data.high, period=20)[-1]:
                self.buy()
                self.trailing_stop = self.data.close - (self.atr[0] * self.params.atr_multiplier)
                self.is_long = True
        else:
            new_stop = self.data.close - (self.atr[0] * self.params.atr_multiplier)
            self.trailing_stop = max(self.trailing_stop, new_stop)
            if self.data.close < self.trailing_stop:
                self.sell()
                self.is_long = False

class OpeningRangeBreakout(bt.Strategy):
    params = (('minutes', 15),)
    def __init__(self):
        pass
    def next(self):
        pass

# --- 6. STRATEGY DISPATCHER "REGISTRY" ---
STRATEGY_REGISTRY = {
    "SmaCrossStrategy": SmaCross,
    "RsiStrategy": RsiStrategy,
    "MultiInstrumentSignal": MultiInstrumentSignal,
    "BollingerBandsReversion": BollingerBandsReversion,
    "MACDStrategy": MACDStrategy,
    "StochasticStrategy": StochasticStrategy,
    "DonchianChannelBreakout": DonchianChannelBreakout,
    "EmaCrossStrategy": EmaCrossStrategy,
    "ATRTrailingStopStrategy": ATRTrailingStopStrategy,
    "OpeningRangeBreakout": OpeningRangeBreakout,
}

# --- 7. CORE LOGIC (DATA & BACKTESTING) ---
def get_historical_data(symbol: str, start_date: str, end_date: str) -> pd.DataFrame:
    """ Fetches historical data from Fyers and formats it for Backtrader. """
    print(f"Fetching data for {symbol} from {start_date} to {end_date}...")
    try:
        historical_data = {
            "symbol": symbol, "resolution": "D", "date_format": "1",
            "range_from": start_date, "range_to": end_date, "cont_flag": "1"
        }
        response = fyers.history(data=historical_data)
        
        if not response:
            print(f"No response from Fyers for {symbol}.")
            return pd.DataFrame()

        if response.get("s") != "ok":
            print(f"Error fetching data for {symbol}: {response.get('message')}")
            return pd.DataFrame()
        
        # --- 1. GET THE LIST OF CANDLES ---
        candles = response.get('candles', [])
        
        # --- 2. CHECK IF THE LIST IS EMPTY (MOVED UP) ---
        if not candles:
            print(f"No candle data returned for {symbol}.")
            return pd.DataFrame()

        # --- 3. THIS IS THE CRITICAL FIX ---
        # Assign column names as defined by Fyers API v3 response
        df = pd.DataFrame(
            candles, 
            columns=['timestamp', 'open', 'high', 'low', 'close', 'volume']
        )
        # --- END OF FIX ---

        # The rest of your code will now work correctly
        if 'timestamp' in df.columns:
            df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
        elif 'date' in df.columns:
            df['datetime'] = pd.to_datetime(df['date'])
        else:
            # This 'else' block will likely not be needed anymore, 
            # but is fine as a fallback.
            df['datetime'] = pd.to_datetime(df.index)

        df.set_index('datetime', inplace=True)

        # Best-effort rename (fallbacks)
        colmap = {}
        if 'open' in df.columns: colmap['open'] = 'Open'
        if 'high' in df.columns: colmap['high'] = 'High'
        if 'low' in df.columns: colmap['low'] = 'Low'
        if 'close' in df.columns: colmap['close'] = 'Close'
        if 'volume' in df.columns: colmap['volume'] = 'Volume'
        if colmap:
            df.rename(columns=colmap, inplace=True)

        # Ensure we only return OHLCV
        wanted = ['Open', 'High', 'Low', 'Close', 'Volume']
        available = [c for c in wanted if c in df.columns]
        df = df[available].sort_index()
        
        # This check is now more accurate.
        if 'Open' not in df.columns:
             print(f"Data fetched for {symbol}, but 'Open' column is missing after processing.")
             return pd.DataFrame()
             
        print(f"Successfully fetched {len(df)} rows for {symbol}.")
        return df
    except Exception as e:
        print(f"Exception in get_historical_data: {e}")
        return pd.DataFrame()

def run_backtest_internal(strategy_id: str, data_feeds: List[pd.DataFrame], params_dict: Dict[str, Any]) -> str:
    """ Internal Backtrader runner. """
    
    StrategyClass = STRATEGY_REGISTRY.get(strategy_id)
    if not StrategyClass: return f"Error: Strategy '{strategy_id}' not found."
    if not data_feeds or all(df.empty for df in data_feeds): return "Error: No data provided."

    cerebro = bt.Cerebro()
    for df in data_feeds:
        if not df.empty:
            cerebro.adddata(bt.feeds.PandasData(dataname=df))

    cerebro.addstrategy(StrategyClass, **(params_dict or {}))
    
    start_cash = 100000.0
    cerebro.broker.setcash(start_cash)
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    cerebro.broker.setcommission(commission=0.001)

    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade')
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

    print(f"--- Running Backtest: {strategy_id} ---")
    results = cerebro.run()
    strategy_instance = results[0]
    
    final_value = cerebro.broker.getvalue()
    trade_analysis = strategy_instance.analyzers.trade.get_analysis() or {}
    sharpe_analysis = strategy_instance.analyzers.sharpe.get_analysis() or {}
    drawdown_analysis = strategy_instance.analyzers.drawdown.get_analysis() or {}

    total_trades = trade_analysis.get('total', {}).get('total', 0) if isinstance(trade_analysis, dict) else 0
    win_rate = (trade_analysis.get('won', {}).get('total', 0) / total_trades) * 100 if total_trades > 0 else 0
    
    sharpe_val = sharpe_analysis.get('sharperatio', 'N/A')
    # drawdown_analysis format can vary; handle safely
    drawdown_pct = 'N/A'
    try:
        if isinstance(drawdown_analysis, dict):
            drawdown_pct = drawdown_analysis.get('max', {}).get('drawdown', 'N/A')
        else:
            drawdown_pct = getattr(drawdown_analysis, 'max', {}).get('drawdown', 'N/A')
    except Exception:
        drawdown_pct = 'N/A'
    
    result_str = (
        f"Backtest for '{strategy_id}' Complete.\n"
        f"  Final Portfolio Value: ₹{final_value:,.2f}\n"
        f"  Total Return: {((final_value - start_cash) / start_cash) * 100:,.2f}%\n"
        f"  Sharpe Ratio: {sharpe_val}\n"
        f"  Max. Drawdown: {drawdown_pct}\n"
        f"  Total Trades: {total_trades}\n"
        f"  Win Rate: {win_rate:,.2f}%"
    )
    print(result_str)
    return result_str

# --- 8. LANGCHAIN AGENT TOOLS (@tool) ---
# Initialize retriever globally
retriever = get_retriever()


@tool
def strategy_search(query: str) -> dict:
    """
    Searches the strategy database to find the best strategy_id and its
    required parameters based on a user's description (e.g., 'golden cross').
    """
    print(f"--- RAG: Searching for query: '{query}' ---")
    try:
        # Use modern retriever invocation if available, fallback to get_relevant_documents
        results = None
        try:
            results = retriever.invoke(query)
        except Exception:
            try:
                results = retriever.get_relevant_documents(query)
            except Exception:
                results = retriever.similarity_search(query, k=1)
        if not results:
            return {"error": "No matching strategy found."}
        
        doc = results[0]
        metadata = dict(doc.metadata) if hasattr(doc, "metadata") else dict(doc)
        if isinstance(metadata.get("parameters"), str):
            metadata["parameters"] = json.loads(metadata["parameters"])
        
        print(f"--- RAG: Found strategy: {metadata.get('strategy_id', 'UNKNOWN')} ---")
        return metadata
    except Exception as e:
        return {"error": f"Error during strategy search: {e}"}

@tool
def run_strategy_backtest(
    strategy_id: str, 
    symbols: List[str], 
    start_date: str, 
    end_date: str, 
    params_dict: Dict[str, Any]
) -> str:
    """
    Runs the final backtest. This tool should only be called when all 
    information (strategy_id, symbols, dates, and parameters) is collected.
    """
    print(f"--- Tool: run_strategy_backtest called for {strategy_id} ---")
    
    data_feeds = []
    for symbol in symbols:
        df = get_historical_data(symbol, start_date, end_date)
        if df.empty:
            return f"Error: Could not fetch data for symbol {symbol}."
        data_feeds.append(df)
        
    result_str = run_backtest_internal(
        strategy_id=strategy_id,
        data_feeds=data_feeds,
        params_dict=params_dict
    )
    return result_str






# --- Helper to robustly call runnables / agents across versions ---
def invoke_runnable(runnable, input_obj):
    """
    Try common ways to call a LangChain runnable/agent across versions.
    Returns output or raises the last exception.
    """
    last_exc = None
    try:
        # Preferred: modern .invoke API
        return runnable.invoke(input_obj)
    except Exception as e:
        last_exc = e
    try:
        # Fallback: .run
        return runnable.run(input_obj)
    except Exception as e:
        last_exc = e
    try:
        # Fallback: direct call (some runnables are callables)
        return runnable(input_obj)
    except Exception as e:
        last_exc = e
    raise last_exc

# --- 9. AGENT CREATION (Using create_agent) ---
# --- 9. AGENT CREATION (Using create_agent) ---
def create_agent_runnable():
    """Creates the main LangChain tool-calling agent (runnable)."""

    llm = ChatGroq(
        model="openai/gpt-oss-20b",
        temperature=0,
        api_key=GROQ_API_KEY
    )

    # Tools are the python functions decorated with @tool above
    tools = [strategy_search, run_strategy_backtest]

    # ✅ Use `system_prompt` instead of custom ChatPromptTemplate
    system_prompt = (
        "You are a helpful and conversational financial analyst chatbot. "
        "Your goal is to help the user backtest trading strategies.\n\n"
        "Here is your workflow:\n"
        "1. Greet the user and engage in general conversation.\n"
        "2. If the user asks to backtest a strategy:\n"
        "   a. First, use the `strategy_search` tool to find the correct `strategy_id` and its required parameters.\n"
        "   b. Show the user the strategy you found and its required parameters.\n"
        "   c. Politely ask for ALL missing information: symbols, start date, end date, and parameter values.\n"
        "   d. For a `MultiInstrumentSignal`, ask for the list of signal symbols AND the single trade symbol.\n"
        "3. Once you have EVERYTHING, call the `run_strategy_backtest` tool ONCE with all collected info and present the backtest results.\n"
    )

    # ✅ New API syntax — `prompt` replaced by `system_prompt`
    agent_runnable = create_agent(
        model=llm,
        tools=tools,
        system_prompt=system_prompt
    )
    return agent_runnable


# --- 10. MAIN CHAT LOOP ---
def main():
    """Main function to run the chatbot."""
    
    print("--- 🤖 HedgeOne Chatbot Initializing... ---")
    
    # Check for missing API keys
    if not (GROQ_API_KEY and FYERS_CLIENT_ID and FYERS_TOKEN):
        print("Error: Missing API keys in .env file.")
        print("Please ensure GROQ_API_KEY, FYERS_CLIENT_ID, and FYERS_TOKEN are set.")
        return

    agent_runnable = create_agent_runnable()
    chat_history = []

    print("Chatbot is ready! Type 'exit' to quit.")
    
    while True:
        try:
            query = input("You: ")
            print(f"You: {query}")
            if query.lower() in ['exit', 'quit']:
                print("Goodbye!")
                break
            
            # Build chat_history Messages for prompt
            agent_history = []
            for msg in chat_history:
                if msg['role'] == 'user':
                    agent_history.append(HumanMessage(content=msg['content']))
                else:
                    agent_history.append(AIMessage(content=msg['content']))

            # Input object to the agent; matches ChatPromptTemplate variables
            input_obj = {
                "messages": agent_history + [HumanMessage(content=query)]
            }

            # Use robust invoker helper (works across langchain releases)
# Use robust invoker helper (works across langchain releases)
            response = agent_runnable.invoke(input_obj) # <-- THIS IS THE 

            # Response handling varies by LLM/agent. We try to extract textual output robustly.
            output = None
            if isinstance(response, dict):
                output = response['messages'][-1].content # <-- THIS IS THE 
            else:
                output = str(response)

            chat_history.append({"role": "user", "content": query})
            chat_history.append({"role": "ai", "content": output})

            print(f"Chatbot: {output}")

        except Exception as e:
            print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


--- 🤖 HedgeOne Chatbot Initializing... ---
Chatbot is ready! Type 'exit' to quit.
You: Hello, I am Rahul
Chatbot: Hello Rahul! 👋 How’s your day going so far? Are you looking into any particular markets or strategies today?
You: Run the SmaCrossStrategy for NSE:TCS-EQ from 2024-01-01 to 2024-12-31 with n1=10 and n2=20
--- RAG: Searching for query: 'SmaCrossStrategy' ---
--- RAG: Found strategy: SmaCrossStrategy ---
--- Tool: run_strategy_backtest called for SmaCrossStrategy ---
Fetching data for NSE:TCS-EQ from 2024-01-01 to 2024-12-31...
Successfully fetched 249 rows for NSE:TCS-EQ.
--- Running Backtest: SmaCrossStrategy ---
Backtest for 'SmaCrossStrategy' Complete.
  Final Portfolio Value: ₹101,154.64
  Total Return: 1.15%
  Sharpe Ratio: None
  Max. Drawdown: 3.6043785212618964
  Total Trades: 6
  Win Rate: 33.33%
Chatbot: Here’s the backtest summary for the **SmaCrossStrategy** on **NSE:TCS‑EQ** from **2024‑01‑01** to **2024‑12‑31** (fast MA = 10, slow MA = 20):

| Metric | Value |


## With Symbol File

In [4]:
# corrected_hedgeone_agent.py
# --- 1. IMPORTS (All in one place) ---
import os
import json
import numpy as np
import pandas as pd
import backtrader as bt
from dotenv import load_dotenv
from typing import List, Dict, Any
from datetime import datetime

# Fyers
from fyers_apiv3 import fyersModel

# LangChain (modern)
from langchain_groq import ChatGroq
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# RAG / Vector Store
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings

# --- 2. API & CONFIGURATION ---
load_dotenv()
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
FYERS_CLIENT_ID = os.environ.get("FYERS_CLIENT_ID")
FYERS_TOKEN = os.environ.get("FYERS_TOKEN")

# Global Fyers Model
fyers = fyersModel.FyersModel(
    client_id=FYERS_CLIENT_ID,
    token=FYERS_TOKEN,
    is_async=False,
    log_path=""
)

# Global RAG Config
STRATEGY_VECTOR_STORE_PATH = "faiss_index_strategies"
SYMBOL_VECTOR_STORE_PATH = "faiss_index_symbols"
SYMBOL_CSV_PATH = "symbols.csv"

# --- 3. RAG METADATA (Replaces strategy_metadata.json) ---
STRATEGY_METADATA_LIST = [
    {
        "strategy_id": "SmaCrossStrategy",
        "description": "A simple trend-following strategy. It buys when a short-term moving average (n1) crosses above a long-term one (n2) and sells on the reverse cross. Best for simple, trending markets. Also known as a 'Golden Cross' or 'Death Cross'.",
        "parameters": [
            {"name": "n1", "type": "int", "description": "The period for the fast moving average, e.g., 50"},
            {"name": "n2", "type": "int", "description": "The period for the slow moving average, e.g., 200"}
        ]
    },
    {
        "strategy_id": "RsiStrategy",
        "description": "A mean-reversion strategy. It buys when the Relative Strength Index (RSI) crosses below an 'oversold' level (e.g., 30) and sells when it crosses above an 'overbought' level (e.g., 70).",
        "parameters": [
            {"name": "period", "type": "int", "description": "The lookback period for the RSI, typically 14."},
            {"name": "oversold", "type": "int", "description": "The RSI level considered oversold, typically 30."},
            {"name": "overbought", "type": "int", "description": "The RSI level considered overbought, typically 70."}
        ]
    },
    {
        "strategy_id": "MultiInstrumentSignal",
        "description": "A complex multi-asset strategy. It trades one target instrument (e.g., Nifty) based on the average performance of a basket of other 'signal' instruments (e.g., 10 stocks). All signal symbols must be provided first, and the target trade symbol last.",
        "parameters": []
    },
    {
        "strategy_id": "BollingerBandsReversion",
        "description": "A mean-reversion strategy. It buys when the price touches or crosses below the lower Bollinger Band and sells when it touches or crosses above the upper Bollinger Band.",
        "parameters": [
            {"name": "period", "type": "int", "description": "The lookback period for the moving average, typically 20."},
            {"name": "devfactor", "type": "float", "description": "The number of standard deviations for the bands, typically 2.0."}
        ]
    },
    {
        "strategy_id": "MACDStrategy",
        "description": "A trend-following strategy based on the Moving Average Convergence Divergence (MACD). It buys when the MACD line crosses above the signal line and sells when it crosses below.",
        "parameters": [
            {"name": "fast_ema", "type": "int", "description": "The period for the fast EMA, typically 12."},
            {"name": "slow_ema", "type": "int", "description": "The period for the slow EMA, typically 26."},
            {"name": "signal_ema", "type": "int", "description": "The period for the signal line EMA, typically 9."}
        ]
    },
    {
        "strategy_id": "StochasticStrategy",
        "description": "A momentum oscillator strategy. It buys when the %K line crosses above the %D line in the oversold region (e.g., below 20) and sells when it crosses below in the overbought region (e.g., above 80).",
        "parameters": [
            {"name": "k_period", "type": "int", "description": "The lookback period for %K, typically 14."},
            {"name": "d_period", "type": "int", "description": "The smoothing period for %D, typically 3."},
            {"name": "oversold", "type": "int", "description": "The oversold level, typically 20."},
            {"name": "overbought", "type": "int", "description": "The overbought level, typically 80."}
        ]
    },
    {
        "strategy_id": "DonchianChannelBreakout",
        "description": "A trend-following breakout strategy (like Turtle Trading). It buys when the price breaks above the upper channel (N-period high) and sells when it breaks below the lower channel (N-period low).",
        "parameters": [
            {"name": "period", "type": "int", "description": "The lookback period for the channel, typically 20."}
        ]
    },
    {
        "strategy_id": "EmaCrossStrategy",
        "description": "A simple trend-following strategy using Exponential Moving Averages (EMAs), which are faster to react than SMAs. Buys when the fast EMA (n1) crosses above the slow EMA (n2).",
        "parameters": [
            {"name": "n1", "type": "int", "description": "The period for the fast EMA, e.g., 12."},
            {"name": "n2", "type": "int", "description": "The period for the slow EMA, e.g., 26."}
        ]
    },
    {
        "strategy_id": "ATRTrailingStopStrategy",
        "description": "A trend-following strategy that uses an Average True Range (ATR) based trailing stop-loss. It buys on a signal (e.g., new high) and holds until the price crosses below the trailing stop.",
        "parameters": [
            {"name": "atr_period", "type": "int", "description": "The lookback period for the ATR, typically 14."},
            {"name": "atr_multiplier", "type": "float", "description": "The multiplier for the ATR value, e.g., 3.0."}
        ]
    },
    {
        "strategy_id": "OpeningRangeBreakout",
        "description": "An intraday strategy. It buys if the price breaks above the high of the first N minutes (e.g., 15) and sells/shorts if it breaks below the low. (Note: Requires intraday data).",
        "parameters": [
            {"name": "minutes", "type": "int", "description": "The opening range period in minutes, e.g., 15 or 30."}
        ]
    }
]

# --- 4. RAG SETUP FUNCTIONS ---
def create_strategy_vector_store():
    """
    Creates and saves a FAISS vector store from the strategy metadata.
    """
    print("Loading strategy metadata...")
    documents = []
    for strategy in STRATEGY_METADATA_LIST:
        content = f"Strategy: {strategy['strategy_id']}. Description: {strategy['description']}"
        metadata = {
            "strategy_id": strategy['strategy_id'],
            "description": strategy['description'],
            "parameters": json.dumps(strategy['parameters'])
        }
        documents.append(Document(page_content=content, metadata=metadata))
    
    print("Creating strategy embeddings... (This may take a moment on first run)")
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vector_store = FAISS.from_documents(documents, embeddings)
    
    vector_store.save_local(STRATEGY_VECTOR_STORE_PATH)
    print(f"Strategy vector store saved to {STRATEGY_VECTOR_STORE_PATH}")

def get_strategy_retriever():
    """
    Loads the saved strategy FAISS vector store as a retriever.
    """
    if not os.path.exists(STRATEGY_VECTOR_STORE_PATH):
        print("Strategy vector store not found. Creating a new one...")
        create_strategy_vector_store()
        
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    # allow_dangerous_deserialization kept for compatibility with older saved indexes
    vector_store = FAISS.load_local(STRATEGY_VECTOR_STORE_PATH, embeddings, allow_dangerous_deserialization=True)
    return vector_store.as_retriever(search_kwargs={"k": 1})

def create_symbol_vector_store():
    """
    Creates and saves a FAISS vector store from the symbols.csv file.
    """
    if not os.path.exists(SYMBOL_CSV_PATH):
        print(f"Error: '{SYMBOL_CSV_PATH}' not found.")
        print("Please create this file with 'Company Name,Symbol' columns.")
        raise FileNotFoundError(f"Required file not found: {SYMBOL_CSV_PATH}")

    print(f"Loading {SYMBOL_CSV_PATH}...")
    try:
        df = pd.read_csv(SYMBOL_CSV_PATH)
        if 'Company Name' not in df.columns or 'Symbol' not in df.columns:
            print("Error: symbols.csv must have 'Company Name' and 'Symbol' columns.")
            raise ValueError("Invalid symbols.csv format")
    except Exception as e:
        print(f"Error reading symbols.csv: {e}")
        raise

    documents = []
    for _, row in df.iterrows():
        # Ensure data is string and clean
        company_name = str(row['Company Name']).strip()
        symbol = str(row['Symbol']).strip()
        
        if not company_name or not symbol:
            continue # Skip empty rows

        content = f"Company Name: {company_name}, Symbol: {symbol}"
        metadata = {
            "company_name": company_name,
            "symbol": symbol
        }
        documents.append(Document(page_content=content, metadata=metadata))
    
    if not documents:
        print(f"No valid data found in {SYMBOL_CSV_PATH}.")
        raise ValueError(f"No documents to process from {SYMBOL_CSV_PATH}")

    print("Creating symbol embeddings... (This may take a moment)")
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vector_store = FAISS.from_documents(documents, embeddings)
    
    vector_store.save_local(SYMBOL_VECTOR_STORE_PATH)
    print(f"Symbol vector store saved to {SYMBOL_VECTOR_STORE_PATH}")

def get_symbol_retriever():
    """
    Loads the saved symbol FAISS vector store as a retriever.
    """
    if not os.path.exists(SYMBOL_VECTOR_STORE_PATH):
        print("Symbol vector store not found. Creating a new one...")
        create_symbol_vector_store()
        
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vector_store = FAISS.load_local(SYMBOL_VECTOR_STORE_PATH, embeddings, allow_dangerous_deserialization=True)
    # Returns top 3 matches as requested
    return vector_store.as_retriever(search_kwargs={"k": 3})


# --- 5. BACKTESTING STRATEGY CLASSES (copy your classes exactly) ---
class SmaCross(bt.Strategy):
    params = (('n1', 20), ('n2', 50))
    def __init__(self):
        self.sma_fast = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.n1)
        self.sma_slow = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.n2)
        self.crossover = bt.indicators.CrossOver(self.sma_fast, self.sma_slow)
    def next(self):
        if not self.position and self.crossover > 0: self.buy()
        elif self.position and self.crossover < 0: self.sell()

class RsiStrategy(bt.Strategy):
    params = (('period', 14), ('oversold', 30), ('overbought', 70))
    def __init__(self):
        self.rsi = bt.indicators.RSI(self.data.close, period=self.params.period)
    def next(self):
        if not self.position and self.rsi < self.params.oversold: self.buy()
        elif self.position and self.rsi > self.params.overbought: self.sell()

class MultiInstrumentSignal(bt.Strategy):
    def __init__(self):
        self.trade_data = self.datas[-1]
        self.signal_datas = self.datas[:-1]
    def next(self):
        returns = []
        for d in self.signal_datas:
            if len(d) > 1:
                daily_return = (d.close[0] - d.close[-1]) / d.close[-1]
                returns.append(daily_return)
        if not returns: return
        avg_return = np.mean(returns)
        if avg_return < -0.005 and not self.getposition(self.trade_data): self.sell(data=self.trade_data)
        elif avg_return > 0.005 and not self.getposition(self.trade_data): self.buy(data=self.trade_data)
        elif abs(avg_return) < 0.001 and self.getposition(self.trade_data): self.close(data=self.trade_data)

class BollingerBandsReversion(bt.Strategy):
    params = (('period', 20), ('devfactor', 2.0))
    def __init__(self):
        self.bbands = bt.indicators.BollingerBands(self.data.close, period=self.params.period, devfactor=self.params.devfactor)
    def next(self):
        if not self.position and self.data.close < self.bbands.lines.bot: self.buy()
        elif self.position and self.data.close > self.bbands.lines.top: self.sell()

class MACDStrategy(bt.Strategy):
    params = (('fast_ema', 12), ('slow_ema', 26), ('signal_ema', 9))
    def __init__(self):
        self.macd = bt.indicators.MACD(self.data.close, period_me1=self.params.fast_ema, period_me2=self.params.slow_ema, period_signal=self.params.signal_ema)
        self.crossover = bt.indicators.CrossOver(self.macd.macd, self.macd.signal)
    def next(self):
        if not self.position and self.crossover > 0: self.buy()
        elif self.position and self.crossover < 0: self.sell()

class StochasticStrategy(bt.Strategy):
    params = (('k_period', 14), ('d_period', 3), ('oversold', 20), ('overbought', 80))
    def __init__(self):
        self.stoch = bt.indicators.Stochastic(self.data, period=self.params.k_period, period_d=self.params.d_period)
    def next(self):
        if not self.position and self.stoch.percK < self.params.oversold and self.stoch.percD < self.params.oversold and self.stoch.percK > self.stoch.percD:
            self.buy()
        elif self.position and self.stoch.percK > self.params.overbought and self.stoch.percD > self.params.overbought and self.stoch.percK < self.stoch.percD:
            self.sell()

class DonchianChannelBreakout(bt.Strategy):
    params = (('period', 20),)
    def __init__(self):
        self.donchian = bt.indicators.DonchianChannels(self.data, period=self.params.period)
    def next(self):
        if not self.position and self.data.close[0] > self.donchian.dch[0]: self.buy()
        elif self.position and self.data.close[0] < self.donchian.dcl[0]: self.sell()

class EmaCrossStrategy(bt.Strategy):
    params = (('n1', 12), ('n2', 26))
    def __init__(self):
        self.ema_fast = bt.indicators.EMA(self.data.close, period=self.params.n1)
        self.ema_slow = bt.indicators.EMA(self.data.close, period=self.params.n2)
        self.crossover = bt.indicators.CrossOver(self.ema_fast, self.ema_slow)
    def next(self):
        if not self.position and self.crossover > 0: self.buy()
        elif self.position and self.crossover < 0: self.sell()

class ATRTrailingStopStrategy(bt.Strategy):
    params = (('atr_period', 14), ('atr_multiplier', 3.0))
    def __init__(self):
        self.atr = bt.indicators.ATR(self.data, period=self.params.atr_period)
        self.trailing_stop = 0
        self.is_long = False
    def next(self):
        if not self.is_long:
            if self.data.close > bt.indicators.Highest(self.data.high, period=20)[-1]:
                self.buy()
                self.trailing_stop = self.data.close - (self.atr[0] * self.params.atr_multiplier)
                self.is_long = True
        else:
            new_stop = self.data.close - (self.atr[0] * self.params.atr_multiplier)
            self.trailing_stop = max(self.trailing_stop, new_stop)
            if self.data.close < self.trailing_stop:
                self.sell()
                self.is_long = False

class OpeningRangeBreakout(bt.Strategy):
    params = (('minutes', 15),)
    def __init__(self):
        pass
    def next(self):
        pass

# --- 6. STRATEGY DISPATCHER "REGISTRY" ---
STRATEGY_REGISTRY = {
    "SmaCrossStrategy": SmaCross,
    "RsiStrategy": RsiStrategy,
    "MultiInstrumentSignal": MultiInstrumentSignal,
    "BollingerBandsReversion": BollingerBandsReversion,
    "MACDStrategy": MACDStrategy,
    "StochasticStrategy": StochasticStrategy,
    "DonchianChannelBreakout": DonchianChannelBreakout,
    "EmaCrossStrategy": EmaCrossStrategy,
    "ATRTrailingStopStrategy": ATRTrailingStopStrategy,
    "OpeningRangeBreakout": OpeningRangeBreakout,
}

# --- 7. CORE LOGIC (DATA & BACKTESTING) ---
def get_historical_data(symbol: str, start_date: str, end_date: str) -> pd.DataFrame:
    """ Fetches historical data from Fyers and formats it for Backtrader. """
    print(f"Fetching data for {symbol} from {start_date} to {end_date}...")
    try:
        historical_data = {
            "symbol": symbol, "resolution": "D", "date_format": "1",
            "range_from": start_date, "range_to": end_date, "cont_flag": "1"
        }
        response = fyers.history(data=historical_data)
        
        if not response:
            print(f"No response from Fyers for {symbol}.")
            return pd.DataFrame()

        if response.get("s") != "ok":
            print(f"Error fetching data for {symbol}: {response.get('message')}")
            return pd.DataFrame()
        
        # --- 1. GET THE LIST OF CANDLES ---
        candles = response.get('candles', [])
        
        # --- 2. CHECK IF THE LIST IS EMPTY (MOVED UP) ---
        if not candles:
            print(f"No candle data returned for {symbol}.")
            return pd.DataFrame()

        # --- 3. THIS IS THE CRITICAL FIX ---
        # Assign column names as defined by Fyers API v3 response
        df = pd.DataFrame(
            candles, 
            columns=['timestamp', 'open', 'high', 'low', 'close', 'volume']
        )
        # --- END OF FIX ---

        # The rest of your code will now work correctly
        if 'timestamp' in df.columns:
            df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
        elif 'date' in df.columns:
            df['datetime'] = pd.to_datetime(df['date'])
        else:
            # This 'else' block will likely not be needed anymore, 
            # but is fine as a fallback.
            df['datetime'] = pd.to_datetime(df.index)

        df.set_index('datetime', inplace=True)

        # Best-effort rename (fallbacks)
        colmap = {}
        if 'open' in df.columns: colmap['open'] = 'Open'
        if 'high' in df.columns: colmap['high'] = 'High'
        if 'low' in df.columns: colmap['low'] = 'Low'
        if 'close' in df.columns: colmap['close'] = 'Close'
        if 'volume' in df.columns: colmap['volume'] = 'Volume'
        if colmap:
            df.rename(columns=colmap, inplace=True)

        # Ensure we only return OHLCV
        wanted = ['Open', 'High', 'Low', 'Close', 'Volume']
        available = [c for c in wanted if c in df.columns]
        df = df[available].sort_index()
        
        # This check is now more accurate.
        if 'Open' not in df.columns:
             print(f"Data fetched for {symbol}, but 'Open' column is missing after processing.")
             return pd.DataFrame()
             
        print(f"Successfully fetched {len(df)} rows for {symbol}.")
        return df
    except Exception as e:
        print(f"Exception in get_historical_data: {e}")
        return pd.DataFrame()

def run_backtest_internal(strategy_id: str, data_feeds: List[pd.DataFrame], params_dict: Dict[str, Any]) -> str:
    """ Internal Backtrader runner. """
    
    StrategyClass = STRATEGY_REGISTRY.get(strategy_id)
    if not StrategyClass: return f"Error: Strategy '{strategy_id}' not found."
    if not data_feeds or all(df.empty for df in data_feeds): return "Error: No data provided."

    cerebro = bt.Cerebro()
    for df in data_feeds:
        if not df.empty:
            cerebro.adddata(bt.feeds.PandasData(dataname=df))

    cerebro.addstrategy(StrategyClass, **(params_dict or {}))
    
    start_cash = 100000.0
    cerebro.broker.setcash(start_cash)
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    cerebro.broker.setcommission(commission=0.001)

    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade')
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

    print(f"--- Running Backtest: {strategy_id} ---")
    results = cerebro.run()
    strategy_instance = results[0]
    
    final_value = cerebro.broker.getvalue()
    trade_analysis = strategy_instance.analyzers.trade.get_analysis() or {}
    sharpe_analysis = strategy_instance.analyzers.sharpe.get_analysis() or {}
    drawdown_analysis = strategy_instance.analyzers.drawdown.get_analysis() or {}

    total_trades = trade_analysis.get('total', {}).get('total', 0) if isinstance(trade_analysis, dict) else 0
    win_rate = (trade_analysis.get('won', {}).get('total', 0) / total_trades) * 100 if total_trades > 0 else 0
    
    sharpe_val = sharpe_analysis.get('sharperatio', 'N/A')
    # drawdown_analysis format can vary; handle safely
    drawdown_pct = 'N/A'
    try:
        if isinstance(drawdown_analysis, dict):
            drawdown_pct = drawdown_analysis.get('max', {}).get('drawdown', 'N/A')
        else:
            drawdown_pct = getattr(drawdown_analysis, 'max', {}).get('drawdown', 'N/A')
    except Exception:
        drawdown_pct = 'N/A'
    
    result_str = (
        f"Backtest for '{strategy_id}' Complete.\n"
        f"  Final Portfolio Value: ₹{final_value:,.2f}\n"
        f"  Total Return: {((final_value - start_cash) / start_cash) * 100:,.2f}%\n"
        f"  Sharpe Ratio: {sharpe_val}\n"
        f"  Max. Drawdown: {drawdown_pct}\n"
        f"  Total Trades: {total_trades}\n"
        f"  Win Rate: {win_rate:,.2f}%"
    )
    print(result_str)
    return result_str

# --- 8. LANGCHAIN AGENT TOOLS (@tool) ---
# Initialize retrievers globally
strategy_retriever = get_strategy_retriever()
symbol_retriever = get_symbol_retriever()


@tool
def strategy_search(query: str) -> dict:
    """
    Searches the strategy database to find the best strategy_id and its
    required parameters based on a user's description (e.g., 'golden cross').
    """
    print(f"--- RAG: Searching for query: '{query}' ---")
    try:
        # Use modern retriever invocation if available, fallback to get_relevant_documents
        results = None
        try:
            results = strategy_retriever.invoke(query)
        except Exception:
            try:
                results = strategy_retriever.get_relevant_documents(query)
            except Exception:
                results = strategy_retriever.similarity_search(query, k=1)
        if not results:
            return {"error": "No matching strategy found."}
        
        doc = results[0]
        metadata = dict(doc.metadata) if hasattr(doc, "metadata") else dict(doc)
        if isinstance(metadata.get("parameters"), str):
            metadata["parameters"] = json.loads(metadata["parameters"])
        
        print(f"--- RAG: Found strategy: {metadata.get('strategy_id', 'UNKNOWN')} ---")
        return metadata
    except Exception as e:
        return {"error": f"Error during strategy search: {e}"}

@tool
def symbol_search(company_name_query: str) -> List[Dict[str, str]]:
    """
    Searches the stock symbol database for a company name.
    Returns the top 3 potential matches (company name and symbol).
    """
    print(f"--- Symbol RAG: Searching for query: '{company_name_query}' ---")
    try:
        # Use modern retriever invocation
        results = symbol_retriever.invoke(company_name_query)
        
        matches = []
        if not results:
             print("--- Symbol RAG: No matches found. ---")
             return []
             
        for doc in results:
            # Extract from metadata, which is cleaner
            name = doc.metadata.get('company_name', 'N/A')
            symbol = doc.metadata.get('symbol', 'N/A')
            if name != 'N/A' and symbol != 'N/A':
                matches.append({"company_name": name, "symbol": symbol})
        
        print(f"--- Symbol RAG: Found matches: {matches} ---")
        return matches
    except Exception as e:
        print(f"--- Symbol RAG: Error: {e} ---")
        return [{"error": f"Error during symbol search: {e}"}]

@tool
def run_strategy_backtest(
    strategy_id: str, 
    symbols: List[str], 
    start_date: str, 
    end_date: str, 
    params_dict: Dict[str, Any]
) -> str:
    """
    Runs the final backtest. This tool should only be called when all 
    information (strategy_id, symbols, dates, and parameters) is collected.
    """
    print(f"--- Tool: run_strategy_backtest called for {strategy_id} ---")
    
    data_feeds = []
    for symbol in symbols:
        df = get_historical_data(symbol, start_date, end_date)
        if df.empty:
            return f"Error: Could not fetch data for symbol {symbol}."
        data_feeds.append(df)
        
    result_str = run_backtest_internal(
        strategy_id=strategy_id,
        data_feeds=data_feeds,
        params_dict=params_dict
    )
    return result_str






# --- Helper to robustly call runnables / agents across versions ---
def invoke_runnable(runnable, input_obj):
    """
    Try common ways to call a LangChain runnable/agent across versions.
    Returns output or raises the last exception.
    """
    last_exc = None
    try:
        # Preferred: modern .invoke API
        return runnable.invoke(input_obj)
    except Exception as e:
        last_exc = e
    try:
        # Fallback: .run
        return runnable.run(input_obj)
    except Exception as e:
        last_exc = e
    try:
        # Fallback: direct call (some runnables are callables)
        return runnable(input_obj)
    except Exception as e:
        last_exc = e
    raise last_exc

# --- 9. AGENT CREATION (Using create_agent) ---
def create_agent_runnable():
    """Creates the main LangChain tool-calling agent (runnable)."""

    llm = ChatGroq(
        model="openai/gpt-oss-120b", # Using a more powerful model for better tool use
        temperature=0,
        api_key=GROQ_API_KEY
    )

    # Tools are the python functions decorated with @tool above
    tools = [strategy_search, symbol_search, run_strategy_backtest]

    # ✅ Use `system_prompt` instead of custom ChatPromptTemplate
    system_prompt = (
        "You are a helpful and conversational financial analyst chatbot. "
        "Your goal is to help the user backtest trading strategies.\n\n"
        "Here is your workflow:\n"
        "1. Greet the user.\n"
        "2. If the user asks to backtest a strategy (e.g., 'backtest a golden cross on Reliance'):\n"
        "   a. First, use the `strategy_search` tool to find the `strategy_id` and its parameters (e.g., from 'golden cross').\n"
        "   b. Next, check if the user provided an exact symbol (like 'NSE:RELIANCE-EQ') or a company name (like 'Reliance').\n"
        "   c. **If they provided a company name**, you MUST use the `symbol_search` tool to find the top 3 matches.\n"
        "   d. **Show these 3 matches** to the user (e.g., 'I found: 1. Reliance Industries (NSE:RELIANCE-EQ), ...') and ask them to confirm which symbol they want to use. You cannot proceed without their confirmation.\n"
        "   e. Politely ask for ALL other missing information: the exact symbol (if not confirmed), start date, end date, and any strategy parameter values (like n1, n2).\n"
        "   f. For `MultiInstrumentSignal`, ask for the list of signal symbols AND the single trade symbol.\n"
        "3. **Only when you have 100% of the information** (the exact `strategy_id`, all exact `symbols`, `start_date`, `end_date`, and `params_dict`) call the `run_strategy_backtest` tool ONCE.\n"
        "4. Present the results clearly."
    )


    # ✅ New API syntax — `prompt` replaced by `system_prompt`
    agent_runnable = create_agent(
        model=llm,
        tools=tools,
        system_prompt=system_prompt
    )
    return agent_runnable


# --- 10. MAIN CHAT LOOP ---
def main():
    """Main function to run the chatbot."""
    
    print("--- 🤖 HedgeOne Chatbot Initializing... ---")
    
    # Check for missing API keys
    if not (GROQ_API_KEY and FYERS_CLIENT_ID and FYERS_TOKEN):
        print("Error: Missing API keys in .env file.")
        print("Please ensure GROQ_API_KEY, FYERS_CLIENT_ID, and FYERS_TOKEN are set.")
        return

    # Initialize RAG retrievers
    print("Initializing retrievers...")
    try:
        # These functions (defined in Section 8) will create the stores if missing
        _ = strategy_retriever
        _ = symbol_retriever
        print("Retrievers ready.")
    except Exception as e:
        print(f"Error initializing retrievers: {e}")
        print("Please ensure 'symbols.csv' exists in the same directory and you have write permissions.")
        return
        
    agent_runnable = create_agent_runnable()
    chat_history = []

    print("Chatbot is ready! Type 'exit' to quit.")
    
    while True:
        try:
            query = input("You: ")
            print(f"\n\nYou: {query}")
            if query.lower() in ['exit', 'quit']:
                print("Goodbye!")
                break
            
            # Build chat_history Messages for prompt
            agent_history = []
            for msg in chat_history:
                if msg['role'] == 'user':
                    agent_history.append(HumanMessage(content=msg['content']))
                else:
                    agent_history.append(AIMessage(content=msg['content']))

            # Input object to the agent; matches ChatPromptTemplate variables
            input_obj = {
                "messages": agent_history + [HumanMessage(content=query)]
            }

            # Use robust invoker helper (works across langchain releases)
            response = agent_runnable.invoke(input_obj) # <-- THIS IS THE 

            # Response handling varies by LLM/agent. We try to extract textual output robustly.
            output = None
            if isinstance(response, dict):
                output = response['messages'][-1].content # <-- THIS IS THE 
            else:
                output = str(response)

            chat_history.append({"role": "user", "content": query})
            chat_history.append({"role": "ai", "content": output})

            print(f"Chatbot: {output}")

        except Exception as e:
            print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

--- 🤖 HedgeOne Chatbot Initializing... ---
Initializing retrievers...
Retrievers ready.
Chatbot is ready! Type 'exit' to quit.


You: hello
Chatbot: Hello! 👋 How can I assist you today with backtesting a trading strategy?


You: what all you can do for me
Chatbot: I’m your personal **financial‑analysis assistant** – here to help you explore, test, and understand trading ideas. Here’s a quick rundown of what I can do for you:

| What you might want to do | How I can help |
|---------------------------|----------------|
| **Back‑test a trading strategy** (e.g., moving‑average cross, RSI, MACD, custom signal combos) | I’ll locate the right strategy template, gather the required parameters, fetch the correct ticker(s), run the back‑test over the dates you choose, and present the performance metrics (returns, drawdowns, win‑rate, equity curve, etc.). |
| **Find the right ticker** for a company name or partial symbol | I can search the symbol database, show you the top 3 matches, and let you

In [1]:
print("Hello")

Hello
